In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256/'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 300
start_epoch = 0
train_batch = 180
test_batch = 180
lr = 0.1
schedule = [20, 125, 225, 275]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style2/128/b1' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
#     transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4, nesterov=True)
# optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

# Train

In [13]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [1 | 300] LR: 0.100000
1/431 Data:1.783 | Batch:6.230 | Total:0:00:06 | ETA:0:44:40 | Loss:0.693012535572052 | top1:55.00000762939453
11/431 Data:0.002 | Batch:0.680 | Total:0:00:13 | ETA:0:08:54 | Loss:3.566168324513869 | top1:50.65656661987305
21/431 Data:0.002 | Batch:0.666 | Total:0:00:20 | ETA:0:04:47 | Loss:3.6547159211976186 | top1:51.11111068725586
31/431 Data:0.001 | Batch:0.687 | Total:0:00:27 | ETA:0:04:36 | Loss:2.756077595295445 | top1:51.37992477416992
41/431 Data:0.001 | Batch:0.680 | Total:0:00:34 | ETA:0:04:29 | Loss:2.259211393391214 | top1:51.19240951538086
51/431 Data:0.002 | Batch:0.688 | Total:0:00:40 | ETA:0:04:20 | Loss:1.956159588168649 | top1:51.557735443115234
61/431 Data:0.001 | Batch:0.693 | Total:0:00:47 | ETA:0:04:17 | Loss:1.7523081996401801 | top1:51.67577362060547
71/431 Data:0.001 | Batch:0.689 | Total:0:00:54 | ETA:0:04:10 | Loss:1.607059445179684 | top1:51.59624481201172
81/431 Data:0.000 | Batch:0.672 | Total:0:01:01 | ETA:0:04:00 | Loss:1.

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


44/44 Data:0.002 | Batch:1.122 | Total:0:00:14 | ETA:0:00:00 | Loss:0.6934639485982749 | top1:50.0

Epoch: [2 | 300] LR: 0.170000
1/431 Data:1.324 | Batch:2.118 | Total:0:00:02 | ETA:0:15:11 | Loss:0.6963841915130615 | top1:56.11111831665039
11/431 Data:0.002 | Batch:0.709 | Total:0:00:09 | ETA:0:05:55 | Loss:0.7078761024908586 | top1:50.55555725097656
21/431 Data:0.001 | Batch:0.689 | Total:0:00:16 | ETA:0:04:42 | Loss:0.7039263645807902 | top1:51.587303161621094
31/431 Data:0.001 | Batch:0.669 | Total:0:00:21 | ETA:0:03:37 | Loss:0.7019353297448927 | top1:51.9713249206543
41/431 Data:0.002 | Batch:0.683 | Total:0:00:28 | ETA:0:04:31 | Loss:0.7035727355538345 | top1:51.50406265258789
51/431 Data:0.002 | Batch:0.692 | Total:0:00:35 | ETA:0:04:20 | Loss:0.7033403188574547 | top1:51.753814697265625
61/431 Data:0.001 | Batch:0.682 | Total:0:00:40 | ETA:0:03:35 | Loss:0.701011417342014 | top1:52.28597640991211
71/431 Data:0.002 | Batch:0.684 | Total:0:00:47 | ETA:0:04:06 | Loss:0.700648767

271/431 Data:0.006 | Batch:0.674 | Total:0:03:04 | ETA:0:01:51 | Loss:0.6927059787665786 | top1:53.29233169555664
281/431 Data:0.001 | Batch:0.715 | Total:0:03:10 | ETA:0:01:43 | Loss:0.6926763392852294 | top1:53.37287139892578
291/431 Data:0.002 | Batch:0.673 | Total:0:03:17 | ETA:0:01:36 | Loss:0.6926507656926552 | top1:53.379150390625
301/431 Data:0.001 | Batch:0.697 | Total:0:03:24 | ETA:0:01:30 | Loss:0.6926326472497858 | top1:53.364707946777344
311/431 Data:0.002 | Batch:0.690 | Total:0:03:31 | ETA:0:01:23 | Loss:0.6926182606212579 | top1:53.30474853515625
321/431 Data:0.002 | Batch:0.676 | Total:0:03:38 | ETA:0:01:16 | Loss:0.692628310105511 | top1:53.34718322753906
331/431 Data:0.001 | Batch:0.658 | Total:0:03:45 | ETA:0:01:08 | Loss:0.6925388248665455 | top1:53.38704299926758
341/431 Data:0.002 | Batch:0.688 | Total:0:03:52 | ETA:0:01:03 | Loss:0.6925310649829876 | top1:53.39035415649414
351/431 Data:0.002 | Batch:0.690 | Total:0:03:58 | ETA:0:00:56 | Loss:0.6925363153473943 |

111/431 Data:0.001 | Batch:0.698 | Total:0:01:18 | ETA:0:03:43 | Loss:0.6848910166336609 | top1:55.63563537597656
121/431 Data:0.002 | Batch:0.678 | Total:0:01:25 | ETA:0:03:33 | Loss:0.6811824504994164 | top1:56.16620635986328
131/431 Data:0.001 | Batch:0.690 | Total:0:01:32 | ETA:0:03:28 | Loss:0.6765786109079841 | top1:56.96352767944336
141/431 Data:0.002 | Batch:0.674 | Total:0:01:39 | ETA:0:03:20 | Loss:0.6718774640813787 | top1:57.659576416015625
151/431 Data:0.001 | Batch:0.668 | Total:0:01:45 | ETA:0:03:12 | Loss:0.6645103637745838 | top1:58.61663055419922
161/431 Data:0.001 | Batch:0.674 | Total:0:01:52 | ETA:0:03:05 | Loss:0.6571274742206431 | top1:59.44789505004883
171/431 Data:0.005 | Batch:0.712 | Total:0:01:59 | ETA:0:02:59 | Loss:0.6464039036753582 | top1:60.503570556640625
181/431 Data:0.000 | Batch:0.687 | Total:0:02:05 | ETA:0:02:20 | Loss:0.638210085544797 | top1:61.38735580444336
191/431 Data:0.002 | Batch:0.693 | Total:0:02:12 | ETA:0:02:46 | Loss:0.628482118639022

...

71/431 Data:0.002 | Batch:0.800 | Total:0:00:53 | ETA:0:04:13 | Loss:0.0033316548556478 | top1:99.89046478271484
81/431 Data:0.006 | Batch:0.708 | Total:0:01:00 | ETA:0:04:14 | Loss:0.0030616613886970162 | top1:99.90398406982422
91/431 Data:0.023 | Batch:0.740 | Total:0:01:07 | ETA:0:04:08 | Loss:0.0028812478218581987 | top1:99.91454315185547
101/431 Data:0.007 | Batch:0.703 | Total:0:01:14 | ETA:0:04:00 | Loss:0.002808425718060312 | top1:99.90650177001953
111/431 Data:0.011 | Batch:0.736 | Total:0:01:21 | ETA:0:03:46 | Loss:0.0028098633822639944 | top1:99.90491485595703
121/431 Data:0.001 | Batch:0.714 | Total:0:01:29 | ETA:0:03:47 | Loss:0.0026168103697483645 | top1:99.91277313232422
131/431 Data:0.001 | Batch:0.764 | Total:0:01:36 | ETA:0:03:34 | Loss:0.00250607496755107 | top1:99.91519165039062
141/431 Data:0.008 | Batch:0.731 | Total:0:01:43 | ETA:0:03:25 | Loss:0.002360392297169768 | top1:99.92121124267578
151/431 Data:0.001 | Batch:0.565 | Total:0:01:49 | ETA:0:02:56 | Loss:0.00

341/431 Data:0.002 | Batch:0.773 | Total:0:04:04 | ETA:0:01:05 | Loss:0.003134148677882207 | top1:99.8924789428711
351/431 Data:0.004 | Batch:0.796 | Total:0:04:12 | ETA:0:01:00 | Loss:0.0032218229268756136 | top1:99.89080047607422
361/431 Data:0.001 | Batch:0.778 | Total:0:04:19 | ETA:0:00:51 | Loss:0.0031854536319565908 | top1:99.8938217163086
371/431 Data:0.003 | Batch:0.739 | Total:0:04:26 | ETA:0:00:45 | Loss:0.003148243322014097 | top1:99.89517974853516
381/431 Data:0.002 | Batch:0.702 | Total:0:04:34 | ETA:0:00:37 | Loss:0.0031502916611452388 | top1:99.89502716064453
391/431 Data:0.001 | Batch:0.692 | Total:0:04:41 | ETA:0:00:30 | Loss:0.0030863632305029214 | top1:99.897705078125
401/431 Data:0.002 | Batch:0.730 | Total:0:04:48 | ETA:0:00:22 | Loss:0.003200744556122578 | top1:99.89471435546875
411/431 Data:0.001 | Batch:0.790 | Total:0:04:55 | ETA:0:00:15 | Loss:0.00320372435029476 | top1:99.89456939697266
421/431 Data:0.018 | Batch:0.792 | Total:0:05:03 | ETA:0:00:08 | Loss:0.0

171/431 Data:0.001 | Batch:0.517 | Total:0:02:02 | ETA:0:02:48 | Loss:0.0037451985092409506 | top1:99.88953399658203
181/431 Data:0.005 | Batch:0.705 | Total:0:02:08 | ETA:0:02:26 | Loss:0.004290146911132486 | top1:99.87416076660156
191/431 Data:0.002 | Batch:0.665 | Total:0:02:15 | ETA:0:02:52 | Loss:0.004289946548894586 | top1:99.87202453613281
201/431 Data:0.002 | Batch:0.689 | Total:0:02:22 | ETA:0:02:43 | Loss:0.004256637924241044 | top1:99.87285614013672
211/431 Data:0.011 | Batch:0.774 | Total:0:02:30 | ETA:0:02:48 | Loss:0.004229495222289921 | top1:99.87625122070312
221/431 Data:0.001 | Batch:0.699 | Total:0:02:37 | ETA:0:02:30 | Loss:0.004386079353202668 | top1:99.8692855834961
231/431 Data:0.003 | Batch:0.739 | Total:0:02:44 | ETA:0:02:20 | Loss:0.004456375003798645 | top1:99.8629150390625
241/431 Data:0.016 | Batch:0.743 | Total:0:02:51 | ETA:0:02:15 | Loss:0.004532473679073783 | top1:99.86168670654297
251/431 Data:0.003 | Batch:0.699 | Total:0:02:59 | ETA:0:02:15 | Loss:0.0

1/431 Data:2.831 | Batch:3.496 | Total:0:00:03 | ETA:0:25:04 | Loss:0.00012748771405313164 | top1:100.00001525878906
11/431 Data:0.003 | Batch:0.746 | Total:0:00:10 | ETA:0:07:03 | Loss:0.007276541261076504 | top1:99.5959701538086
21/431 Data:0.008 | Batch:0.820 | Total:0:00:18 | ETA:0:05:04 | Loss:0.004594559907369937 | top1:99.76191711425781
31/431 Data:0.001 | Batch:0.522 | Total:0:00:24 | ETA:0:04:20 | Loss:0.0035664440628381507 | top1:99.82079315185547
41/431 Data:0.002 | Batch:0.697 | Total:0:00:31 | ETA:0:04:24 | Loss:0.0036036817455613167 | top1:99.83740234375
51/431 Data:0.002 | Batch:0.703 | Total:0:00:37 | ETA:0:04:04 | Loss:0.0038137023223623378 | top1:99.82571411132812
61/431 Data:0.011 | Batch:0.421 | Total:0:00:43 | ETA:0:03:28 | Loss:0.003850720494319532 | top1:99.83607482910156
71/431 Data:0.001 | Batch:0.710 | Total:0:00:50 | ETA:0:04:25 | Loss:0.003994438831873146 | top1:99.835693359375
81/431 Data:0.003 | Batch:0.780 | Total:0:00:58 | ETA:0:04:15 | Loss:0.0037986832

271/431 Data:0.003 | Batch:0.724 | Total:0:03:13 | ETA:0:01:58 | Loss:0.003769433155279552 | top1:99.87085723876953
281/431 Data:0.027 | Batch:0.738 | Total:0:03:20 | ETA:0:01:49 | Loss:0.003794645999758786 | top1:99.87149810791016
291/431 Data:0.005 | Batch:0.704 | Total:0:03:27 | ETA:0:01:45 | Loss:0.003748081220454725 | top1:99.87210083007812
301/431 Data:0.001 | Batch:0.732 | Total:0:03:34 | ETA:0:01:33 | Loss:0.003675851145561203 | top1:99.87449645996094
311/431 Data:0.003 | Batch:0.713 | Total:0:03:42 | ETA:0:01:27 | Loss:0.0037512168345502743 | top1:99.87496185302734
321/431 Data:0.001 | Batch:0.690 | Total:0:03:49 | ETA:0:01:19 | Loss:0.0038000036558140694 | top1:99.87367248535156
331/431 Data:0.003 | Batch:0.690 | Total:0:03:56 | ETA:0:01:10 | Loss:0.0037652071846052556 | top1:99.87244415283203
341/431 Data:0.002 | Batch:0.770 | Total:0:04:03 | ETA:0:01:06 | Loss:0.0036973272100936953 | top1:99.87455749511719
351/431 Data:0.001 | Batch:0.723 | Total:0:04:10 | ETA:0:01:00 | Los